In [45]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [46]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [47]:
# Define database and collection
db = client.mars_db
collection = db.items

In [48]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/melissa/.wdm/drivers/chromedriver/mac64/89.0.4389.23]


In [49]:
# URL of page to be scraped
# This is not the given website, but I get returns, so I'm working with it for now
news_url = ("https://redplanetscience.com/#")
browser.visit(news_url)

In [50]:
# HTML Object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

In [51]:
# Retrieve the latest element that contains news title and news_paragraph
title = soup.find('div', class_='content_title').text
article_teaser = soup.find('div', class_='article_teaser_body').text

# Display scrapped data 
print(title)
print(article_teaser)

# I think my issue was using find_all and .text together incorrectly and also not having the splinter set up.
# I'm still unsure about how it all relates and why it won't work without the other, but at least I can start moving forward

NASA's Perseverance Rover Will Look at Mars Through These 'Eyes'
A pair of zoomable cameras will help scientists and rover drivers with high-resolution color images.


In [56]:
# JPL Mars image

jpl_url = ("https://spaceimages-mars.com/")
browser.visit(jpl_url)

# Set up new instance of HTML object
html = browser.html

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
soup

<html class=""><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
<!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<title>Space Image</title>
<style type="text/css">.fancybox-margin{margin-right:15px;}</style></head>
<body>
<div class="header">
<nav class="navbar navbar-expand-lg">
<a class="navbar-brand" href="#"><img id="logo" src="image/nasa.png"/><span class="logo">Jet Propulsion Laboratory</span>
<span class="logo1">California Institute of Technology</span></a>
<button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-target="#navbarNav" data-toggle="collapse" type="button

In [57]:
# Gather featured_image_url
# Yea, nothing is woring
# img-card-top

img_url = soup.find('href', class_='showimg')

print(img_url)

None


In [ ]:
https://spaceimages-mars.com/image/featured/mars1.jpg

In [36]:
# # pull image link
# pic_src = []
# for image in images:
#     pic = image['data-fancybox-href']
#     pic_src.append(pic)

# featured_image_url = 'https://www.jpl.nasa.gov' + pic
# featured_image_url

# <a target="_blank" class="showimg fancybox-thumbs" href="image/featured/mars3.jpg"> 
# <button class="btn btn-outline-light"> FULL IMAGE</button></a>

Mars Facts
https://galaxyfacts-mars.com/


Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.


Use Pandas to convert the data to a HTML table string.

In [52]:
# Let's try some table reading
facts_url = 'https://galaxyfacts-mars.com/'

tables = pd.read_html(facts_url)
tables

mars_facts = tables[0]
mars_facts

mars_html = mars_facts.to_html('mars_facts.html')
mars_html

Mars Hemispheres


Visit the astrogeology site here to obtain high resolution images for each of Mars's hemispheres.


You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.


Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.


Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [53]:
# Collect the image urls for the given website
hemisphere_url = 'https://marshemispheres.com/'
browser.visit(hemisphere_url)

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

In [44]:
browser.quit()

In [33]:
# Retrieve page with the requests module
response = requests.get(jpl_url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'html.parser')
# soup


# pull images from website
images = soup.find_all('a', class_="fancybox")
#print(images)